In [1]:
import os
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import BlockBlobService
import json
import time
from datetime import datetime

Get all the variables from cluster's environment variables. The variables should be pushed to the cluster via DevOps pipline in a ideal scenario

In [3]:
# Get Environment varaibles
KeyVault_Scope = os.environ['Azure_KeyVault_Scope']
KeyVault_ADLSGen2_Access_Secret_Name = os.environ['KeyVault_ADLSGen2_Access_Secret_Name']
ADLSGen2_URL = os.environ['ADLSGen2_URL']
ADLSGen2_FileSystem = os.environ['ADLSGen2_FileSystem']
KeyVault_BlobStorage_Access_Secret_Name = os.environ['KeyVault_BlobStorage_Access_Secret_Name']
BlobStorage_URL = os.environ['BlobStorage_URL']
BlobStorage_Output_Container = os.environ['BlobStorage_Output']
Scan_Depth=os.environ['Scan_Depth']
AAD_Client_Id=os.environ['AAD_Client_Id']
KeyVault_Client_Secret_Secret_Name=os.environ['KeyVault_Client_Secret_Secret_Name']
ADLSGen2_Resource_Group=os.environ['ADLSGen2_Resource_Group']
ADLSGen2_Subscription_Id=os.environ['ADLSGen2_Subscription_Id']
AAD_Tenant_Id=os.environ['AAD_Tenant_Id']

Set spart configuration to access ADLS Gen2 file system. This python job using ADLS storage keys to access the file system from Spark.

In [5]:
spark.conf.set(
  "fs.azure.account.key."+ADLSGen2_URL,
  dbutils.secrets.get(scope = KeyVault_Scope, key =KeyVault_ADLSGen2_Access_Secret_Name ))

Set spark configuration to access Blob storage to store the output JSON file

In [7]:
# Set spark configuration for output Blob Storage Account
spark.conf.set(
  "fs.azure.account.key."+BlobStorage_URL,
 dbutils.secrets.get(scope = KeyVault_Scope, key = KeyVault_BlobStorage_Access_Secret_Name))

Method to upload create file in Azure blob storage. <br>
Inputs : Content, file name

In [9]:
def uploadtoBlob(content, file_name):
  try:
    timenow = datetime.now()
    file_name = file_name+str(timenow.strftime("-%m%d%Y-%H-%M-%S"))+".json"
    result = dbutils.fs.put("wasbs://"+BlobStorage_Output_Container+"@"+BlobStorage_URL+"/"+file_name,content,True)
    if result == True:
      print("File creation success!")
    else:
      print("File creation failed")
  except Exception as e:
    print('Error occurred while creating blob', e)

Method to make JSON in a needed format <br>
Input - Python list <br>
Output - Json String

In [11]:
def makeoutputjson_storageaccount(name,subscriptionId,resourceGroupName,location,createTime,SKU):
  entity_json ={
    "entity_type_name":"azure_storage_account",
    "created_by":"harvester",
     "attributes":[
         {
             "attr_name":"qualifiedname",
             "attr_value":""
         },
         {
            "attr_name":"name",
            "attr_value":name
        },
        {
            "attr_name":"subscriptionId",
            "attr_value":subscriptionId
        }, 
        {
            "attr_name":"resourceGroupName",
            "attr_value":resourceGroupName
        },
        {
            "attr_name":"location",
            "attr_value":location
        },
        {
            "attr_name":"createTime",
            "attr_value":createTime
        },
        {
            "attr_name":"accessTier",
            "attr_value":"Unknown"
        },
        {
            "attr_name":"SKU",
            "attr_value":SKU
        },
        {
            "attr_name":"kind",
            "attr_value":"StorageV2"
        }
     ]
  }
  json_string= json.dumps(entity_json)
  return json_string

In [12]:
def makeoutputjson_filesystem(name):
  entity_json ={
      "entity_type_name": "azure_datalake_gen2_filesystem",
      "created_by": "harvester",
      "attributes": [{
          "attr_name": "qualifiedName",
          "attr_value": "",
          "is_entityref": False
       }, 
        {
          "attr_name": "name",
          "attr_value": name,
          "is_entityref": False
      }
      ]
    }
  json_string= json.dumps(entity_json)
  return json_string
  

In [13]:
def makeoutputjson(entitylist):
  entity_final=[]
  for entity in entitylist:
    entity_json ={
      "entity_type_name": "azure_datalake_gen2_resource_set",
      "created_by": "harvester",
      "attributes": [{
          "attr_name": "qualifiedName",
          "attr_value": "",
          "is_entityref": False
       }, 
        {
          "attr_name": "name",
          "attr_value": entity,
          "is_entityref": False
      }
      ]
    }
    entity_final.append(entity_json)
  json_string= json.dumps(entity_final)
  return json_string

Method to scan the ADLS Gen 2 file system folders recursively using databricks dbutils

In [15]:
def getpath(path, level, entitylist, root_path ):  
  files = dbutils.fs.ls(path)
  for file in files:    
    pathvalue = str(file.path)      
    pathvalue_string = pathvalue.split(root_path)      
    pathvalue_entity =pathvalue_string[-1]    
    entitylist.append(pathvalue_entity)
    if level <= int(Scan_Depth):
        newlevel= level+1        
        getpath(file.path,newlevel,entitylist,root_path)
  return entitylist

In [16]:
 
def scanfilesystem(filesystem_name, account_url):  
  entitylist=[]
  startlevel =1
  try:    
    root_path = "abfss://"+filesystem_name+"@"+account_url+"/"
    entitylist = getpath(root_path,startlevel,entitylist,root_path)
  except:
    print('Error in scan file system')
  finally:
    return entitylist

<b>Main method</b> <br>
This script is using Azure serice principal to get access to the storage account properties <br>
Service principal client id is stored in environment variable and secrets are pulled from Azure KeyVault <br><br>
This script doesn't mount ADLS Gen 2 file system to databricks, instead it directly access the file system

In [18]:
subscription_id =  ADLSGen2_Subscription_Id
ad_client_id=AAD_Client_Id
ad_client_secret=dbutils.secrets.get(scope = KeyVault_Scope, key =KeyVault_Client_Secret_Secret_Name ) 
ad_tenantid=AAD_Tenant_Id
#
resource_group_name=ADLSGen2_Resource_Group
storage_account_name=ADLSGen2_URL[0:ADLSGen2_URL.find('.')]
#Make credential object
credentials = ServicePrincipalCredentials(client_id=ad_client_id, secret=ad_client_secret, tenant=ad_tenantid)    
resource_client = ResourceManagementClient(credentials, subscription_id)
storage_client = StorageManagementClient(credentials, subscription_id)
storage_account = storage_client.storage_accounts.get_properties(resource_group_name, storage_account_name)
# Get properties of storage account
# Get properties of storage account
sa_creation_time=storage_account.creation_time.strftime("%Y-%m-%d %H:%M:%S")
sa_kind=storage_account.kind
sa_location=storage_account.location
sa_name=storage_account.name
sa_sku=storage_account.sku.name
print(sa_sku)
# Go further only for Storage Gen 2
if sa_kind =='StorageV2':
  # make output json for storage account
  output_json_sa = makeoutputjson_storageaccount(sa_name,subscription_id,resource_group_name,sa_location,sa_creation_time,sa_sku)
  print(output_json_sa)
  #Get Storage Account file system properties
  storage_keys = storage_client.storage_accounts.list_keys(resource_group_name, storage_account_name)
  storage_keys = {v.key_name: v.value for v in storage_keys.keys}
  block_blob_service = BlockBlobService(account_name=storage_account_name, account_key=storage_keys['key1'])
  containers = block_blob_service.list_containers()
  filesystems=[]
  # Get file system names
  for container in containers:
      filesystems.append(container.name)
  for filesystem in filesystems:  
    # Pull the entity
    # make output json for file system
    output_json_fs= makeoutputjson_filesystem(filesystem)    
    #print(output_json_fs)
    entitylist= scanfilesystem(filesystem,ADLSGen2_URL)
    if len(entitylist) >0:
      output_json= makeoutputjson(entitylist)
      output_json_filesystem={
        "azure_storage_account":json.loads(output_json_sa),
        "azure_datalake_gen2_filesystem":json.loads(output_json_fs),
        "azure_datalake_gen2_resource_set":json.loads(output_json)
      }
      json_string_final= json.dumps(output_json_filesystem)
      output_filename=filesystem+"@"+ADLSGen2_URL
      uploadtoBlob(json_string_final,output_filename)

Standard_RAGRS
{"created_by": "harvester", "entity_type_name": "azure_storage_account", "attributes": [{"attr_value": "", "attr_name": "qualifiedname"}, {"attr_value": "beamdatav2", "attr_name": "name"}, {"attr_value": "25d668e3-ce02-4784-9a44-924bd741f07a", "attr_name": "subscriptionId"}, {"attr_value": "walmart-gbs", "attr_name": "resourceGroupName"}, {"attr_value": "southeastasia", "attr_name": "location"}, {"attr_value": "2019-07-08 03:47:05", "attr_name": "createTime"}, {"attr_value": "Unknown", "attr_name": "accessTier"}, {"attr_value": "Standard_RAGRS", "attr_name": "SKU"}, {"attr_value": "StorageV2", "attr_name": "kind"}]}
Wrote 9076 bytes.
File creation success!
Wrote 9077 bytes.
File creation success!
Wrote 9088 bytes.
File creation success!
Wrote 321537 bytes.
File creation success!
Wrote 755152 bytes.
File creation success!
Wrote 378056 bytes.
File creation success!
Wrote 396789 bytes.
File creation success!
Wrote 1068095 bytes.
File creation success!